In [1]:
import copy

import pandas
from scipy.io import arff
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
from dl85 import DL85Predictor
from dl85 import DL85Classifier
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import numpy as np
import MyCode.utils as utils
import graphviz
from MyCode import relabeling
from MyCode import relabeling_sklearn

In [2]:
df = pandas.read_csv("dataset_perso/Lawsuit.csv")
FILE_NAME = "lawsuit_k_supp2_relab_sklearn"

del df['ID']
salary_mean = np.mean(np.array(df["Sal94"].tolist(), float))
for i in range(0,len(df)):
    if df.at[i, "Sal94"] >= salary_mean:
        df.at[i, "Salary_mean"] = 1
    else:
        df.at[i, "Salary_mean"] = 0
del df["Sal94"]
del df["Sal95"]
col = ['Rank', 'Dept']
df = df.drop(['Prate', 'Exper'], axis=1)
df = pd.get_dummies(df, columns=col)

for i in range(0,len(df)):
    if df.at[i, "Gender"] == 1:
        df.at[i, "Gender"] = 0
    else:
        df.at[i, "Gender"] = 1


X = df.loc[:, ~df.columns.isin(['Gender', 'Salary_mean'])]
y = df['Salary_mean']
sensitive = df['Gender']
X_train, X_test, y_train, y_test, sensitive_train, sensitive_test = utils.train_test_split(1, X, y, sensitive)

In [3]:
pd.set_option('max_columns', None)

# src: https://www.kaggle.com/hjmjerry/gender-discrimination

for leaf_limit in range (1, 4):
    df['discrimination_relab_'+str(leaf_limit)] = np.nan
    df['accuracy_relab_'+str(leaf_limit)] = np.nan

for threshold in range (1, 4):
    df['discrimination_relab_'+str(threshold)] = np.nan
    df['accuracy_relab_'+str(threshold)] = np.nan
df

,Gender,Clin,Cert,Salary_mean,Rank_1,Rank_2,Rank_3,Dept_1,Dept_2,Dept_3,Dept_4,Dept_5,Dept_6,discrimination_relab_1,accuracy_relab_1,discrimination_relab_2,accuracy_relab_2,discrimination_relab_3,accuracy_relab_3
0,0,0,0,0.0,0,0,1,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,0,0.0,0,1,0,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,0.0,1,0,0,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,0,1,1,1.0,0,0,1,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,0,0.0,0,0,1,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,1,1,1,1.0,0,1,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN
257,1,1,1,1.0,0,0,1,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN
258,1,1,1,1.0,1,0,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN
259,1,1,1,1.0,1,0,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = pd.read_csv('save/'+FILE_NAME+".csv")
df = df.sort_values(['depth', 'random_state'])
df

,k,depth,random_state,accuracy_train,accuracy_test,discrimination_train,discrimination_train_pred,discrimination_test,min_supp,clf.tree_,discrimination_relab_1,accuracy_relab_1,discrimination_relab_2,accuracy_relab_2,discrimination_relab_3,accuracy_relab_3,discrimination_relab_0,accuracy_relab_0,discrimination_relab_0.1,accuracy_relab_0.1,discrimination_relab_0.2,accuracy_relab_0.2,discrimination_relab_0.3,accuracy_relab_0.3
0,0,1,1,0.755981,0.730769,0.325619,0.156476,0.266667,2,<sklearn.tree._tree.Tree object at 0x7f41c1b20...,-0.156476,0.2440,-0.156476,0.2440,-0.156476,0.2440,0.000000,0.6028,0.000000,0.6028,0.156476,0.7559,0.156476,0.7559
1,0,1,3,0.770335,0.673077,0.304364,0.234535,-0.046083,2,<sklearn.tree._tree.Tree object at 0x7f41c1b20...,-0.234535,0.2296,-0.234535,0.2296,-0.234535,0.2296,0.000000,0.6076,0.000000,0.6076,0.000000,0.6076,0.234535,0.7703
2,0,1,5,0.765550,0.692308,0.324563,0.179566,0.172619,2,<sklearn.tree._tree.Tree object at 0x7f41c1b20...,-0.179566,0.2344,-0.179566,0.2344,-0.179566,0.2344,0.000000,0.6076,0.000000,0.6076,0.179566,0.7655,0.179566,0.7655
3,0,1,7,0.755981,0.730769,0.290782,0.185791,0.137931,2,<sklearn.tree._tree.Tree object at 0x7f41c1b20...,-0.185791,0.2440,-0.185791,0.2440,-0.185791,0.2440,0.000000,0.5837,0.000000,0.5837,0.185791,0.7559,0.185791,0.7559
4,0,1,9,0.712919,0.711538,0.316116,0.113636,-0.101307,2,<sklearn.tree._tree.Tree object at 0x7f41c1b20...,-0.113636,0.2870,-0.113636,0.2870,-0.113636,0.2870,0.000000,0.6124,0.000000,0.6124,0.113636,0.7129,0.113636,0.7129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,0,7,89,0.913876,0.865385,0.318990,0.351884,0.205397,2,<sklearn.tree._tree.Tree object at 0x7f41c0f11...,-0.521515,0.2344,-0.521515,0.2344,-0.521515,0.2344,-0.046950,0.6411,-0.046950,0.6411,0.147351,0.8086,0.299866,0.8995
311,0,7,93,0.923445,0.903846,0.298554,0.253099,0.392157,2,<sklearn.tree._tree.Tree object at 0x7f41c0f11...,-0.566116,0.2248,-0.566116,0.2248,-0.566116,0.2248,-0.121901,0.6698,0.073347,0.8325,0.073347,0.8325,0.253099,0.9234
312,0,7,95,0.918660,0.923077,0.353778,0.276616,0.221691,2,<sklearn.tree._tree.Tree object at 0x7f41c0f16...,-0.576880,0.2200,-0.576880,0.2200,-0.576880,0.2200,-0.120313,0.6985,-0.120313,0.6985,0.166572,0.8660,0.276616,0.9186
313,0,7,97,0.933014,0.865385,0.342636,0.272384,0.307692,2,<sklearn.tree._tree.Tree object at 0x7f41c0f11...,-0.510174,0.2440,-0.510174,0.2440,-0.510174,0.2440,-0.171415,0.6028,0.092636,0.8277,0.145640,0.8612,0.272384,0.9330


In [5]:
import math
"""
depth=3
random_state=19
"""
""""
depth=4
random_state=1
"""
depth=4
random_state=39
def round_down(a, exp):
    dec = 10**20
    return math.floor(a * dec)/float(dec)


In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
X_train, X_test, y_train, y_test, sensitive_train, sensitive_test = utils.train_test_split(random_state, X, y, sensitive)

clf = DecisionTreeClassifier(max_depth=depth, random_state=0)

print(f"### Depth: {depth} state: {random_state} ###")
clf.fit(X_train, list(y_train))
y_pred_train = clf.predict(X_train)
accuracy_train = round_down(accuracy_score(y_train, y_pred_train), 8)
print("Accuracy DL8.5 on training set =", accuracy_train)
y_pred_test = clf.predict(X_test)
accuracy_test = round_down(accuracy_score(y_test, y_pred_test), 8)
print("Accuracy DL8.5 on test set =", accuracy_test)
discrimination_train = round_down(utils.discrimination(y_train, sensitive_train), 8)
print("Discrimination of classifier on train", discrimination_train)
discrimination_train_pred = round_down(utils.discrimination(y_pred_train, sensitive_train), 8)
print("Discrimination of classifier on train(pred)", discrimination_train_pred)
discrimination_test = round_down(utils.discrimination(y_pred_test, sensitive_test), 8)
print("Discrimination of classifier on test", discrimination_test)
clf_original = copy.deepcopy(clf)
# RELAB
for leaf_limit in range (1, 4):
    clf = copy.deepcopy(clf_original)
    leafs = list()
    cnt = np.unique(sensitive, return_counts=True)[1]
    leafs_relab = relabeling_sklearn.relab_leaf_limit(clf.tree_, X_train, y_train, y_pred_train, sensitive_train, leaf_limit)
    for leaf in leafs_relab:
        relabeling_sklearn.browse_and_relab(clf, leaf.node_id)

    # NEW PREDICT
    y_pred_train_relab = clf.predict(X_train)


for threshold in [0.3, 0.2, 0.1, 0]:
    clf = copy.deepcopy(clf_original)
    leafs = list()
    cnt = np.unique(sensitive, return_counts=True)[1]
    leafs_relab = relabeling_sklearn.relab(clf.tree_, X_train, y_train, y_pred_train, sensitive_train, threshold)

    for leaf in leafs_relab:
        relabeling_sklearn.browse_and_relab(clf, leaf.node_id)


    # NEW PREDICT
    y_pred_train_relab = clf.predict(X_train)

    if relabeling.discrimination(y_train, y_pred_train_relab, sensitive_train) > threshold:
        print(f"{relabeling.discrimination(y_train, y_pred_train_relab, sensitive_train)} <= {threshold}")
        print()
        sum_acc = 0
        sum_disc = 0
        for leaf in leafs_relab:
            sum_acc += leaf.acc
            sum_disc += leaf.disc
            print(leaf)
            print(leaf.n, leaf.p)
            print(f'value: {leaf.value}')
            print()
        accuracy_train_relab = round_down(accuracy_score(y_train, y_pred_train_relab), 8)
        discrimation_train_pred_relab = round_down(utils.discrimination(y_pred_train_relab, sensitive_train), 8)
        new_acc= round_down(accuracy_train+sum_acc, 8)
        new_disc = round_down(discrimination_train_pred+sum_disc, 8)
        print(f"Accuracy:\n"
              f"    Before      : {accuracy_train}\n"
              f"    Leafs       : {sum_acc}\n"
              f"    After       : {accuracy_train_relab}\n"
              f"    Expected    : {new_acc}\n"
              f"    Check       : {new_acc == accuracy_train_relab}")
        print(f"Discrim:\n"
              f"    Before      : {discrimination_train_pred}\n"
              f"    Leafs       : {sum_disc }\n"
              f"    After       : {discrimation_train_pred_relab}\n"
              f"    Expected    : {new_disc}\n"
              f"    Check       : {new_disc== discrimation_train_pred_relab}")

        tree.plot_tree(clf)
        assert(relabeling.discrimination(y_train, y_pred_train_relab, sensitive_train) <= threshold)
        assert(new_acc== accuracy_train_relab)
        assert(new_disc== discrimation_train_pred_relab)

### Depth: 4 state: 39 ###
Accuracy DL8.5 on training set = 0.8851674641148325
Accuracy DL8.5 on test set = 0.826923076923077
Discrimination of classifier on train 0.31937984496124033
Discrimination of classifier on train(pred) 0.30610465116279073
Discrimination of classifier on test 0.34615384615384615


In [7]:
clf = DecisionTreeClassifier(max_depth=depth, random_state=0)
clf.fit(X_train, list(y_train))
clf.predict([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]])

array([0., 0., 0., 0.])

In [8]:
p1 = list(clf.predict(X_train))

In [9]:
relabeling_sklearn.browse_and_relab(clf, 11)
clf.predict([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]])

array([1., 1., 1., 1.])

In [10]:
p2 = list(clf.predict(X_train))

In [11]:
p1 == p2

False

In [12]:
clf.tree_.value[11]

array([[3., 4.]])

In [13]:
round_down(utils.discrimination(p1, sensitive_train), 8)

0.30610465116279073